<a href="https://colab.research.google.com/github/Yash-Jaiswal28/Deep-Learning/blob/main/Keras_hyperparamter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np
import pandas as pd

In [44]:
df=pd.read_csv('/content/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [45]:
df.corr()['Outcome'] #Give the correlation of each column with the Outcome column

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [46]:
X = df.iloc[:,:-1].values
y=df.iloc[:,-1].values
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [47]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [48]:
X = scaler.fit_transform(X)
X.shape

(768, 8)

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [76]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [51]:
model=Sequential()
model.add(Dense(units=32,activation='relu',input_dim=8))
model.add(Dense(units=1,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [52]:
model.fit(X_train,y_train, batch_size=32, epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 11ms/step - loss: 0.6268 - accuracy: 0.6759 - val_loss: 0.6044 - val_accuracy: 0.7208
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5813 - accuracy: 0.7003 - val_loss: 0.5575 - val_accuracy: 0.7273
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5496 - accuracy: 0.7231 - val_loss: 0.5284 - val_accuracy: 0.7532
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5275 - accuracy: 0.7329 - val_loss: 0.5072 - val_accuracy: 0.7792
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5109 - accuracy: 0.7557 - val_loss: 0.4903 - val_accuracy: 0.7987
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4978 - accuracy: 0.7638 - val_loss: 0.4791 - val_accuracy: 0.8052
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4877 - accuracy: 0.7687 - val_loss: 0.4706 - val_accuracy: 0.8182
Epoch 8/100


## Finding the Best Optimizer

In [53]:
!pip install keras_tuner

In [54]:
import keras_tuner as kt

In [55]:
def build_model(hp):
  model =Sequential()
  model.add(Dense(units=32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])

  return model


In [56]:
tuner = kt.RandomSearch(build_model,
                         objective='val_accuracy',
                         max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [57]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

In [58]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.8116883039474487

Trial 3 summary
Hyperparameters:
optimizer: adam
Score: 0.8051947951316833

Trial 1 summary
Hyperparameters:
optimizer: adadelta
Score: 0.6883116960525513

Trial 0 summary
Hyperparameters:
optimizer: sgd
Score: 0.649350643157959


In [59]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [60]:
model = tuner.get_best_models(num_models=1)[0]

In [61]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [62]:
model.fit(X_train,y_train,batch_size=32,initial_epoch=10,epochs=100,validation_data=(X_test,y_test))

Epoch 11/100
20/20 [==============================] - 1s 12ms/step - loss: 0.4961 - accuracy: 0.7524 - val_loss: 0.4957 - val_accuracy: 0.8117
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4857 - accuracy: 0.7557 - val_loss: 0.4890 - val_accuracy: 0.8182
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4777 - accuracy: 0.7638 - val_loss: 0.4828 - val_accuracy: 0.8182
Epoch 14/100
20/20 [==============================] - 0s 10ms/step - loss: 0.4718 - accuracy: 0.7671 - val_loss: 0.4794 - val_accuracy: 0.8052
Epoch 15/100
20/20 [==============================] - 0s 14ms/step - loss: 0.4679 - accuracy: 0.7704 - val_loss: 0.4766 - val_accuracy: 0.8117
Epoch 16/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4640 - accuracy: 0.7720 - val_loss: 0.4739 - val_accuracy: 0.8117
Epoch 17/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4603 - accuracy: 0.7704 - val_loss: 0.4723 - val_accuracy: 0.8117
Epo

## Number of Nodes

In [63]:
def build_model(hp):
  model=Sequential()
  units= hp.Int('units',min_value=8,max_value=128) #(Lower_limit, upper_limit, Step_size) 8->16->24....

  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

  return model

In [64]:
tuner= kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5,
                       directory="my_dir",
                       project_name="Number of Nodes")

Reloading Tuner from my_dir/Number of Nodes/tuner0.json


In [65]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [66]:
tuner.results_summary()

Results summary
Results in my_dir/Number of Nodes
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
units: 72
Score: 0.8181818127632141

Trial 0 summary
Hyperparameters:
units: 104
Score: 0.8051947951316833

Trial 4 summary
Hyperparameters:
units: 88
Score: 0.7922077775001526

Trial 1 summary
Hyperparameters:
units: 16
Score: 0.7467532753944397

Trial 2 summary
Hyperparameters:
units: 24
Score: 0.7467532753944397


In [67]:
tuner.get_best_hyperparameters()[0].values

{'units': 72}

In [68]:
model=tuner.get_best_models(num_models=1)[0]

In [69]:
model.fit(X_train,y_train,batch_size=32,initial_epoch=6,epochs=100,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 13ms/step - loss: 0.5127 - accuracy: 0.7638 - val_loss: 0.5033 - val_accuracy: 0.8182
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4967 - accuracy: 0.7638 - val_loss: 0.4914 - val_accuracy: 0.8117
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4842 - accuracy: 0.7655 - val_loss: 0.4850 - val_accuracy: 0.8052
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4747 - accuracy: 0.7736 - val_loss: 0.4795 - val_accuracy: 0.8052
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4676 - accuracy: 0.7801 - val_loss: 0.4750 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4629 - accuracy: 0.7801 - val_loss: 0.4740 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4595 - accuracy: 0.7818 - val_loss: 0.4726 - val_accuracy: 0.8052
Epoch 14

## Number of Layers

In [86]:
def build_model(hp):

  model=Sequential()

  counter=0

  for i in range(hp.Int('num_layers ',min_value=1,max_value=10)):

    if counter==0 :
      model.add(Dense(units=hp.Int('units '+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice('activation '+str(i),['relu','tanh','sigmoid']),input_dim=8))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else :
      model.add(Dense(units=hp.Int('units '+str(i),min_value=8,max_value=128,step=8),activation=hp.Choice('activation '+str(i),['relu','tanh','sigmoid'])))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1
  model.add(Dense(1,activation='sigmoid'))

  model.compile(loss='binary_crossentropy',optimizer=hp.Choice('optimizer ',values=['adam','sgd','rmsprop','adadelta','nadam']), metrics=['accuracy'])

  return model


In [87]:
tuner= kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=3,
                       directory="mydir1",
                       project_name="Final1")

Reloading Tuner from mydir1/Final1/tuner0.json


In [88]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [89]:
tuner.get_best_hyperparameters()[0].values

{'num_layers ': 5,
 'units 0': 56,
 'activation 0': 'tanh',
 'dropout0': 0.4,
 'optimizer ': 'nadam',
 'units 1': 8,
 'activation 1': 'relu',
 'dropout1': 0.1,
 'units 2': 8,
 'activation 2': 'relu',
 'dropout2': 0.1,
 'units 3': 8,
 'activation 3': 'relu',
 'dropout3': 0.1,
 'units 4': 8,
 'activation 4': 'relu',
 'dropout4': 0.1}

In [90]:
model=tuner.get_best_models(num_models=1)[0]

In [91]:
model.fit(X_train,y_train,initial_epoch=5,epochs=200,validation_data=(X_test,y_test))

Epoch 6/200
20/20 [==============================] - 3s 14ms/step - loss: 0.5511 - accuracy: 0.7085 - val_loss: 0.5081 - val_accuracy: 0.7597
Epoch 7/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5421 - accuracy: 0.7248 - val_loss: 0.4979 - val_accuracy: 0.7792
Epoch 8/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5426 - accuracy: 0.7085 - val_loss: 0.4980 - val_accuracy: 0.7922
Epoch 9/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5239 - accuracy: 0.7280 - val_loss: 0.4952 - val_accuracy: 0.8052
Epoch 10/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5343 - accuracy: 0.7199 - val_loss: 0.4929 - val_accuracy: 0.8052
Epoch 11/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5265 - accuracy: 0.7166 - val_loss: 0.4905 - val_accuracy: 0.8182
Epoch 12/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5275 - accuracy: 0.7345 - val_loss: 0.4893 - val_accuracy: 0.8182
Epoch 13/